# Análise Comparativa


Essa etapa tem como objetivo comparar alguns modelos para encontrar um que possa ser melhor utilizado dentro do problema, para isso antes realizamos a preparação e o pré-processamento dos dados.

## Preparação dos dados

In [57]:
from IPython.display import Markdown
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, RobustScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import cross_validate, train_test_split, KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor

In [58]:
# Ler o csv
df_day = pd.read_csv('../data/raw/day.csv')
#df_day = pd.read_csv('day.csv')

# Ler o dicionário de dados
df_dict = pd.read_csv('../data/external/dicionario_dados_day.csv')
#df_dict = pd.read_csv('dicionario_dados_day.csv')
display (df_day.head (2))

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801


### Tratamento dos dados

Aqui realizamos a normalização, codificação e o tratamento de dados discrepantes e/ou faltantes dentro do conjunto de dados. Através do dicionário de dados apresentado no notebook/01_exploratory_data_analysis, é possível perceber que as colunas *casual* e *registered* são deviradas da nossa variável alvo, ou seja, a soma das duas é equivalente ao resultado da coluna **cnt**. Portanto, serão excluídos durante o tratamento e não seguirão para o experimento. A coluna *dteday* também não possui informações relevantes para esta análise.

Foi verificado anteriormente que o dataset não possui valores nulos ou outliers.

Além disso, as colunas categóricas já estão codificadas em sequencias numéricas, enquanto as variáveis contínuas estão calculadas dentro da mesma escala.

In [59]:
df_day = df_day.drop(['casual', 'registered', 'dteday', 'instant'], axis = 1)
df_day.head(2)

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,985
1,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,801


In [60]:
df_day.columns

Index(['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday',
       'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'cnt'],
      dtype='object')

- Tentativa


In [61]:
target_column = 'cnt' #Variável alvo

#Obter uma lista das variáveis discretas
discrete_columns = (
    df_dict
    .query('tipo == "Discreta" and variavel != @target_column')
    .variavel
    .to_list()
)

#Obter uma lista das váriaveis nominais
nominal_columns = (
    df_dict
    .query('tipo == "Nominal"')
    .variavel
    .to_list()
)

#Obter uma lista das váriaveis contínuas
continuous_columns = (
    df_dict
    .query('tipo == "Contínua"')
    .variavel
    .to_list()
)

#Obter uma lista das váriaveis ordinais
ordinal_columns = (
    df_dict
    .query('tipo == "Ordinal"')
    .variavel
    .to_list()
)

In [62]:
#Para cada tipo de variavel, será criado uma pipeline para tratar

discrete_preprocessor = Pipeline([
    # Tratamento de dados discrepantes
    ('missing', SimpleImputer(strategy='mean')), # Tratamento de dados faltantes
    # Seleção de variáveis
    ('normalization', MinMaxScaler()) # Normalização
])

nominal_preprocessor = Pipeline([
    # Tratamento de dados discrepantes
    ('missing', SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes
    ('encoder', OneHotEncoder(sparse=False)), # Codificação de variáveis
    # Seleção de variáveis
    ('normalization', MinMaxScaler()), # Normalização
]),
continuous_preprocessor = Pipeline(steps=[
    # Tratamento de dados discrepantes
    ('missing', KNNImputer(n_neighbors=5)), # Tratamento de dados faltantes
    # Seleção de variáveis
    ('normalization', RobustScaler()) # Normalização
]),
ordinal_preprocessor = Pipeline([
    # Tratamento de dados discrepantes
    ('missing', SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes
    ('encoder', OneHotEncoder(sparse=False)), # Codificação de variáveis
    # Seleção de variáveis
    ('normalization', MinMaxScaler())]) # Normalização

In [63]:
# Transformando os dados utilizando os pré-processadores criados acima
preprocessing = ColumnTransformer([
    ("discrete", discrete_preprocessor, discrete_columns),
    ("nominal", nominal_preprocessor, nominal_columns),
    ("continuous", continuous_preprocessor, continuous_columns),
    ("ordinal", ordinal_preprocessor, ordinal_columns)
])
preprocessing

ColumnTransformer(transformers=[('discrete',
                                 Pipeline(steps=[('missing', SimpleImputer()),
                                                 ('normalization',
                                                  MinMaxScaler())]),
                                 []),
                                ('nominal',
                                 (Pipeline(steps=[('missing',
                                                   SimpleImputer(strategy='most_frequent')),
                                                  ('encoder',
                                                   OneHotEncoder(sparse=False)),
                                                  ('normalization',
                                                   MinMaxScaler())]),),
                                 []),
                                ('continuous',
                                 (Pipeline(steps=[('missing', KNNImputer()),
                                                  ('normalization',
                                                   RobustScaler())]),),
                                 []),
                                ('ordinal',
                                 Pipeline(steps=[('missing',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(sparse=False)),
                                                 ('normalization',
                                                  MinMaxScaler())]),
                                 [])])

## Metodologia

Para o teste será utilizado o método de validação cruzada **k-fold**. Além disso, serão apresentados os seguintes modelos:

- Regressão Linear

- Randon forest

- KNeighborsClassifier

- SVM

Cada um desses algoritmos será testado com diferentes parametros, para que possamos encontrar o melhor modelo. O modelos serão comparados por:

- R2 Score: O R² varia entre 0 e 1, quanto maior o R², mais explicativo é o modelo.

- Mean squared error: Mede a diferença quadrática média entre os valores estimados e o valor real.

- Mean absolute error: É uma medida de erros entre observações pareadas que expressam o mesmo fenômeno

## Configuração do experimento

- Separar o conjunto em treino e teste

In [64]:
X = df_day.drop(columns=[target_column], axis=1)
y = (
    df_day[[target_column]]
    .to_numpy()
    .ravel()
)

In [65]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=.3)

- Realizar teste com os modelos definidos separadamente

In [66]:
regr = RandomForestRegressor(random_state=0)
regr.fit(X_train, y_train)
pred_reg = regr.predict(X_test)

print(
    '\nR2 Score:', r2_score(y_test, pred_reg).round(3),
    '\nMean squared error:', np.sqrt(mean_squared_error(y_test, pred_reg)).round(3),
    '\nMean absolute error:', mean_absolute_error(y_test, pred_reg).round(3),
    )


R2 Score: 0.869 
Mean squared error: 693.986 
Mean absolute error: 459.167


In [67]:
regr = LinearRegression()
regr.fit(X_train, y_train)
pred_reg = regr.predict(X_test)

print(
    '\nR2 Score:', r2_score(y_test, pred_reg).round(3),
    '\nMean squared error:', np.sqrt(mean_squared_error(y_test, pred_reg)).round(3),
    '\nMean absolute error:', mean_absolute_error(y_test, pred_reg).round(3),
    )


R2 Score: 0.805 
Mean squared error: 848.187 
Mean absolute error: 639.731


In [68]:
regr = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p = 2)
regr.fit(X_train, y_train)
pred_reg = regr.predict(X_test)

print(
    '\nR2 Score:', r2_score(y_test, pred_reg).round(3),
    '\nMean squared error:', np.sqrt(mean_squared_error(y_test, pred_reg)).round(3),
    '\nMean absolute error:', mean_absolute_error(y_test, pred_reg).round(3),
    )


R2 Score: 0.217 
Mean squared error: 1697.839 
Mean absolute error: 1279.473


In [69]:
regr = SVC(gamma='auto')
regr.fit(X_train, y_train)
pred_reg = regr.predict(X_test)

print(
    '\nR2 Score:', r2_score(y_test, pred_reg).round(3),
    '\nMean squared error:', np.sqrt(mean_squared_error(y_test, pred_reg)).round(3),
    '\nMean absolute error:', mean_absolute_error(y_test, pred_reg).round(3),
    )


R2 Score: 0.023 
Mean squared error: 1897.208 
Mean absolute error: 1455.927


### Utilizando a validação cruzada **K-Fold**

In [70]:
resultados_lr = []
resultados_random_forest = []
resultados_knn = []
resultados_svm = []


for i in range(5):
  print(i)
  kfold = KFold(n_splits=10, shuffle=True, random_state=i)

  random_forest = RandomForestRegressor(criterion = 'squared_error', min_samples_leaf = 1, min_samples_split=5, n_estimators = 10)
  scores = cross_val_score(random_forest, X, y, cv = kfold)
  resultados_random_forest.append(scores.mean())

  knn = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p = 2)
  scores = cross_val_score(knn, X, y, cv = kfold)
  resultados_knn.append(scores.mean())

  svm = SVC(gamma='auto')
  scores = cross_val_score(svm, X, y, cv = kfold)
  resultados_svm.append(scores.mean())

  lr = LinearRegression()
  scores = cross_val_score(lr, X, y, cv = kfold)
  resultados_lr.append(scores.mean())


0
1
2
3
4


In [71]:
resultados = pd.DataFrame({'Regressão linear': resultados_lr, 'Random forest': resultados_random_forest,
                           'KNN': resultados_knn, 'SVM': resultados_svm})
resultados

,Regressão linear,Random forest,KNN,SVM
0,0.780351,0.865474,0.0,0.0
1,0.783535,0.863368,0.0,0.0
2,0.777794,0.863014,0.0,0.0
3,0.779992,0.868712,0.0,0.0
4,0.788611,0.872359,0.0,0.0


## Resultados e discussão

Através dos testes e da comparação dos valores de R², Erro quadratico médio e Erro absoluto médio, foi possível concluir que o modelo que melhor estima os dados apresentados é o Random forest, seguido da Regressão Linear, estes se saíram melhores em todos os atributos. Os modelos KNN e SVM não trouxeram resultados relevantes mesmo após diferentes testes de configurações.

Recapitulando, o objetivo desse estudo é o de praticar os conhecimentos obtidos durante os meses de participação do bootcamp de Ciência de Dados da Atlântico. O dataset escolhido pela equipe foi o Bike Sharing, que apresenta os aluguéis de bicicletas nos anos de 2011 e 2012 nos EUA. Este dataset para análises e predições dos dados.

Pontos positivos do projeto:
- Não há dados nulos ou outliers.
- O dataset está bem descrito em sua origem.
- Os dados possuem escalas parecidas e fáceis de serem tratadas

Pontos negativos:
- Tempo para realização do projeto
- Dificuldade na interpretação de alguns conceitos
